In [ ]:
import datetime
import json
import ast
import re
from dateutil.parser import parse
from dateutil.tz import tzutc

### Actors Dictionary 

In [ ]:
nouns = []

with open("noundict20180225.json") as f:
    for line in f:
        nouns.append(json.loads(line))

In [ ]:
nouns[1214]
# nouns[12814]['dateStart']['$date']
d = parse(nouns[1214]['dateStart']['$date'])
print(d)
d  > datetime.datetime(1950, 1, 1, 0, 0, tzinfo=tzutc()) 

In [ ]:
d

In [ ]:
nouns_2 = nouns[57:]
nouns_by_coders = [i for i in nouns_2 if i['userName'] != 'existEnglishDictionary']

In [ ]:
nouns_by_coders[10]
nouns[149]
type(nouns[149]['dateEnd'])

In [ ]:
entry_to_CAMEO(nouns_by_coders[10])

In [ ]:
def entry_to_CAMEO(one):
    if one['dateStart'] == None or one['dateStart']==  'na' :
        one['dateStart'] = ''
        one['dateEnd'] = ''
    
    if one['dateStart'] == '' or one['dateEnd'] == '':
        print("Error : No date Info on Sen {0}".format(one['sentenceId']))
        return
    if one['dateEnd'] == 'na' : # throw entry
        return 
    
    if one['dateEnd'] == 'now' :
#         dateStart = datetime.datetime.strptime(one['dateStart']['$date'].split('T')[0], '%Y-%m-%d')
        if (type(one['dateStart']) == str) :
            dateStart = parse(one['dateStart'])
        else :
            dateStart = parse(one['dateStart']['$date'])
            
        date_range = " > " + dateStart.strftime('%Y%m%d')
        
    else :  

#         print(one['dateStart'])
        # a if condition else b
        dateStart = parse(one['dateStart']) if type(one['dateStart']) == str else parse(one['dateStart']['$date'])
        dateEnd  = parse(one['dateEnd']) if type(one['dateEnd']) == str else parse(one['dateEnd']['$date'])


        min_date = datetime.datetime(1950, 1, 1, 0, 0, tzinfo=tzutc() )
        max_date = datetime.datetime(2050, 1, 1, 0, 0 , tzinfo=tzutc())
        

        if dateStart < min_date and dateEnd > max_date:
            date_range = ""
        elif dateStart < min_date and dateEnd < min_date:
            date_range = ""
        elif dateStart > min_date and dateEnd < max_date:
            date_range = "{0}-{1}".format(dateStart.strftime('%Y%m%d'), dateEnd.strftime('%Y%m%d'))
        elif dateStart < min_date:
            date_range = "<{0}".format(dateEnd.strftime('%Y%m%d'))
        elif dateEnd > max_date:
            date_range = ">{0}".format(dateStart.strftime('%Y%m%d'))
        else:
            print("whoops ", dateStart, dateEnd)

        if date_range:
            date_range = " " + date_range
        
        
                                   
    entry = "{0} | [{1}{2}{3}{4}]".format(one['word'], one['countryCode'],
                                           one['firstRoleCode'], one['secondRoleCode'],
                                           date_range)

    return entry


In [ ]:

entries = [entry_to_CAMEO(i) for i in nouns_by_coders]
print(len(entries))
entries = [i for i in entries if i]
# entries

#Removing duplicates while preseving order  ( not the most efficient way)
entries_2 = [] 
[entries_2.append(item) for item in entries if item not in entries_2]
entries_2[:10]


In [ ]:
entries_2[5000]

In [ ]:
#Process Entries to form dictionary 
from collections import deque 
master_queue = deque(entries_2) # queue that holds all entires 
temp_queue = deque() # queue used to hold entries for the same entity 


In [ ]:
#helper method for using a queue
def pop_q(q):
    data = -1
    try: 
        data = q.popleft()
        return data
    except IndexError: 
        return -1

def peek_q(q): 
    data = -1
    try: 
        data = q.popleft()
        q.appendleft(data)
        return data
    except IndexError: 
        return -1


processed_entries = [] 


while True : 
    entry = pop_q(master_queue)
    if entry == -1 : 
        print("Done")
        break
    else : 
        # process (split the entry )
        ent_data = entry.split(' | ')
        
        #Check if it is a single entity (primary phase without dates) example ['مسلحون | [REB]'
        #Just add it 
        if len(ent_data[1].split(' ')) ==1:
            processed_entries.append(entry.replace('|' , ''))
            
        else : 
            # add the entry to the temp stack that should have similar entires with names
            if(len(temp_queue) == 0 ) : # just add it 
                temp_queue.append(entry)
            
            else : # if there are elements in the queue, need to check for matching 
                temp_top = peek_q(temp_queue)
                
                # check if names are the same : push to queue
                temp_ent_data = temp_top.split(' | ')
                if (temp_ent_data[0].strip() == ent_data[0].strip() ):
                    temp_queue.append(entry)
                
                #else : we have a punch of entries for the same person/entity with deifferent roles 
                # process
                else : 
                    master_queue.appendleft(entry)
                    
                    temp_q_data = list(temp_queue)
                    roles =["\t{}".format(item.split(" | ")[1]) for item in temp_q_data]
#                     if (temp_ent_data[0].strip() == 'نوري المالكي'): 
#                         print (roles)
#                         print("**")
#                         print('{}\n'.format(temp_ent_data[0].strip()) + '\n'.join(roles))
                    processed_entries.append('{}\n'.format(temp_ent_data[0].strip()) + '\n'.join(roles))
                    temp_queue.clear() 
                    
            
            

In [ ]:
len(processed_entries)

In [ ]:
with open("arabic_actor_dictionary_v2.txt", "w") as text_file:
    text_file.write('\n'.join(processed_entries))
    

In [ ]:
# queue = deque(["Eric", "John", "Michael"])
# queue.append("Terry")           # Terry arrives
# queue.append("Graham")          # Graham arrives
# x = queue.popleft()
# print("popped ", x )
# print(queue)
# queue.appendleft(x)
# print("left_appended ", x )
# print(queue)
# queue.clear()
# print(len(queue))

In [ ]:

# entries_2 = [i for i in entries_2 if i]
# print("**")
# print(len(entries))
# print(len(entries_2))

# formatted_entries = '\n'.join(entries)
# # formatted_entries =  formatted_entries

# formatted_entries[0:500]

In [ ]:
with open("arabic_actor_dictionary.txt", "w") as text_file:
    text_file.write(formatted_entries)
    
with open("entries_2.txt", "w") as text_file: # entreis 2 test
    text_file.write('\n'.join(entries_2))

### Verbs Dictionary

In [ ]:
verbs = [] 
with open("verbs_dict20180227.json") as vd:
    for line in vd : 
        verbs.append(json.loads(line))

In [ ]:
arabic_verbs  = verbs[14:]
print('len ', len(arabic_verbs))

In [ ]:
arabic_verbs[:3]

In [ ]:
acceptable_formats = ["##()()" , "##**()"  , "##**" , "##()" , "##()**"]
#core idea is to switch the first and last characters   
def verb_not_compound(verb) :
    verb_regx = re.findall("#(.+)#" , verb)
    if len(verb_regx) == 0 : 
        return False 
    if len(verb_regx[0].strip().split(' ')) == 1: 
        return True
    return False 

    
def check_and_reconstruct_verb(verb): 
    verb = " ".join(verb.split()) # to remove all white spaces and duplicate ones 
    if verb == '' :
        return 'ERROR'
    verb_encod =str(verb.encode('ascii', errors='ignore') , 'UTF8').replace(" ", "")
    if verb_encod == '' :  #Lone verb 
        if verb_not_compound(verb):
            return verb
        else :
            return 'ERROR'
#         return ''
    elif verb_encod in acceptable_formats: #verb is already correct(rare cases)- apply regex stright
        ## Check if one word is between ## : True, return verb, false , Corrupt verb
        if verb_not_compound(verb):
            return verb
        else :
            return 'ERROR'
    elif verb_encod =="##" : # Lone verbs that could be wrong : do a circular left shift 
            arr = list(verb)
            if arr[0] =='#' : # it is already in the correct format
                if verb_not_compound(verb): 
                    return verb
                else :
                    return 'ERROR'
            
            arr= arr[-1:] + arr[:-1]
        ## Check if one word is between ## : True, return verb, false , Corrupt verb
            c_verb = "".join(arr)
            if verb_not_compound(c_verb):
                return c_verb
            else :
                return 'ERROR'
            
    else : 
    # switch first and last characters 
        arr = list(verb)
        #replace first char with last - op_1
        #flip last char 
        temp = arr[0]
        arr[0] = arr[len(arr) -1]
        if temp != '*' :
            arr[len(arr) -1] = ')'
        else :
            arr[len(arr) -1] = temp 
            
        #Some verbs are entered inncorretly(coded wrong), at this point we should be able to catch them 
        reconstructed_string = "".join(arr)
        rc_string_encod = str(reconstructed_string.encode('ascii', errors='ignore') , 'UTF8').replace(" ", "").strip()
        if rc_string_encod in acceptable_formats :
        ## Check if one word is between ## : True, return verb, false , Corrupt verb
            if verb_not_compound(reconstructed_string):
                return reconstructed_string
            else : 
                return 'ERROR'
        else :
            return 'ERROR'


Reminder to myself:
    - '#' goes on either side of the verb (just to indicate what should be '*' in English)
    - '*' goes on either side of multi-word direct objects (replaces '{' in English)
    - '(' goes around prepositional phrases (same as English)

In [ ]:
#Fixing the verbs formatting
wrong_verbs =[] 
correct_verbs = []
for verb in arabic_verbs :
    rc_string = check_and_reconstruct_verb(verb['word']) # reconstructed string
    if rc_string == '' : #lone verbs
        print("Orignal and correct Verb : ",  verb )
        verb['word'] = '#{}#'.format(verb['word'])
        print("Lone Verb ==> verb " , verb['word'])
        print("RegEx", re.findall("#(.+)#" , verb['word']))
        correct_verbs.append(verb)
        print("\n************")
    elif rc_string == 'ERROR'  : 
        print("Corrupt Verb (WRONG CODING)\n" , verb['word'])
        wrong_verbs.append(verb)
        print("\n************")
    else :
        print("Orignal Verb : ",  verb )
        print("Reconstructed Verb : " , rc_string)
        print("RegEx", re.findall("#(.+)#" , rc_string))
        verb['word'] = rc_string
        correct_verbs.append(verb)
        
        print("\n************")
    


In [ ]:
# x = re.findall('#(.+)#' , '# هدف fdsa#* برجي مركز التجارة العالمي*')
# print(x[0].strip().split(' '))
# print(len(x[0].strip().split(' ')))
# verb_not_compound('#تفجيرانتحاري#')
# check_and_reconstruct_verb('#تفجير انتحاري#')
# print('Erroneous Verbs Count : ' , str(len(wrong_verbs)))
# print('Correct Verbs Count : ' , str(len(correct_verbs)))
len(correct_verbs)

In [ ]:
#Lemmatize the verbs 
source_verbs = [] 
for i in range(len(correct_verbs)) : #Hoping this will preserve order (it is )
    
    source_verbs.append(re.findall("#(.+)#" , correct_verbs[i]['word'])[0])

len(source_verbs)

In [ ]:
source_verbs[0]

In [ ]:
# make sure to copy the output file to Lemmatizer/Input dir 
with open('verbs_to_lemmatize_v2.txt', 'w') as verbs_file :
    verbs_file.write('\n'.join(source_verbs))


In [ ]:
# Run lemmatizer and get output 
lem_results = []
#had to manually remove last line (empty) from file 
with open('verbs_to_lemmatize.txt.ATB4MT.tok' , 'r') as file : 
    lem_results = file.read()

lem_results = lem_results.split('\n')
len(lem_results)

In [ ]:
print(len(source_verbs))
print(len(lem_results))

In [ ]:
lem_results[3]

In [ ]:
len(source_verbs) == len(lem_results)

In [ ]:
x ='ب+·ب+·بِ+·PRT+·IN+·PREP+ قصف·قصف·قَصْف·NOM·NN·NOUN+CASE_DEF_GEN'
y = x.split(' ')
y

In [ ]:
lem_verbs = []
for i in lem_results : 
#     print(i)
    res = i.split(' ')
    print(res)
    correct_one =''
    for i in res : 
#         if 'NOM' in i : 
#             correct_one = i 
#             break
#         if 'VRB' in i :
#             correct_one = i
#             break 
        if 'PROP' in i  or 'PRT' in i or 'NOM' in i or 'VRB' in i : 
            correct_one = i 
    if correct_one == '' : 
        print('wtf ' , i )
        continue
    lem_verbs.append([correct_one.split('·')[2] ,correct_one.split('·')[0]]) #lemma , source

In [ ]:
lem_verbs[0][0]

In [ ]:
# Put everything back together 
for i in range(len(correct_verbs)) : #Trust its the right order ....
    correct_verbs[i]['verb_lemma'] = lem_verbs[i][0]
# correct_verbs[:10]

In [ ]:
with open('verbs_with_lemmas.json', 'w') as output : 
    for i in correct_verbs : 
        output.write('{}\n'.format(i))

In [ ]:
with open('lemma_verbs_3.txt', 'w') as output : 
    for i in lem_verbs : 
        output.write('{}\t{}\t{}\n'.format(i[0], '-' , 'متعدي'))


In [ ]:
#get the conjugations and put ALL TOGETHER AND DONE 
conj =''
with open('protest_verbs.txt' , 'r') as input_f:
    conj= input_f.read()

# conj = conj.encode('UTF-8')
conj = conj.split('\n')
conj

In [ ]:
total = []
for i in range(len(conj)) : #Trust its the right order ....
    conj_list = eval(eval(conj[i])['conjugations'])
    if conj_list[0] == 'invalid':
        continue
    
#     print(conj_list)
    total.extend(conj_list)

# total   
with open('protest_conjugated.txt', 'w') as output : 
    output.write('\n'.join(total))

In [ ]:
for i in correct_verbs : 
    print (i)

In [ ]:
#check point
with open('complete_verb_date.json', 'w') as output : 
    for i in correct_verbs : 
        output.write('{}\n'.format(i))

In [ ]:
# load the check point
loaded_verbs = []
with open('complete_verb_date.json', 'r') as input_file : 
    for line in input_file : 
        loaded_verbs.append(eval(line))

loaded_verbs[3]

In [ ]:
#Creating the 'dic' 
import re
def create_verb_bock(verb) : 
    #check if verb is lone verb (no patters)-- then 'verbcode' is it's main code, else main code is null

    if verb['verb_conj'][0] == 'invalid' :
        return ''
    
    
    entry = ''
    lone_verb = False 
    if len(verb['word'].split(' ')) == 1 : 
        lone_verb = True
#     print(verb['verbcode'])
    if lone_verb : 
#         entry = '--- {}  [{}] ---\n'.format(re.findall("#(.+)#" , verb['word'])[0] , verb['verbcode'])
        entry = '--- {}  [{}] ---\n'.format(verb['main_verb'], verb['verbcode'])
        entry += '{{ {} }}\n\n'.format(' '.join(verb['verb_conj']))
    
    else  : # not lone verb
#         entry = '--- {}  [---] ---\n'.format(re.findall("#(.+)#" , verb['word'])[0])
        entry = '--- {}  [---] ---\n'.format(verb['main_verb'])
#         entry += '{{ {} }} \n\n'.format(' '.join(verb['verb_conj']))
        entry += '- {}\t\t[{}]\n\n'.format(re.sub(r'#(.+)#' , '* ' , verb['word']), verb['verbcode'])
        
        
        
        
    return entry
        


In [ ]:
create_verb_bock(loaded_verbs[1])

In [ ]:
final_output = ''
for i in loaded_verbs : 
#     print(i)
    final_output += create_verb_bock(i) 


In [ ]:
with open('VERBS.DICT.AR.LEMMA.V3.txt' , 'w') as output: 
    output.write(final_output)

In [ ]:
'{{{}}}'.format('khaled')